<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Julia-version" data-toc-modified-id="Julia-version-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Julia version</a></span><ul class="toc-item"><li><span><a href="#Lambda-expression-(匿名函数)" data-toc-modified-id="Lambda-expression-(匿名函数)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Lambda-expression (匿名函数)</a></span></li><li><span><a href="#Style-guide" data-toc-modified-id="Style-guide-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Style guide</a></span><ul class="toc-item"><li><span><a href="#Writing-type-stable-code-in-julia" data-toc-modified-id="Writing-type-stable-code-in-julia-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Writing type-stable code in julia</a></span></li></ul></li></ul></li></ul></div>

# Julia version

## Julia should be slow?

- You could always write untyped code
- `eval` could load any piece of code into runtime
- A lot of reflections: `typeof`, `methods`, `names`

## But why Julia is fast?

- Design of the language: avoid things that are unoptimizable
- The implementation leverages the choices of the design
- Programming practice

### Design of the language

#### Dynamism in balance

- `eval` is used only global scope
- syntactic macros
- "world age" mechanism

## Lambda-expression (匿名函数)

A _lambda-expression_ can also be considered as the name of a _function_. If `double` is a proper name, like “Michelangelo,” then `x -> x * 2` is a deﬁnite description, like “the man who painted the ceiling of the Sistine Chapel.”

In [1]:
function f1(x)
    x * 2
end

f2(x) = x * 2

f3 = x -> x * 2;  # A symbol `f3` bound to a function

Lambda-expression 告诉我们什么才是一个函数的核心：这个函数所描述的关系或者变化，而不是它的名字。上面三个表达式都可以被抽象出一个公共部分：`x -> x * 2`, 这才是这个函数的核心。

## Style guide

Please refer [YASGuide - Yet Another Style Guide For Julia](https://github.com/jrevels/YASGuide) and [JuMP Style Guide](http://www.juliaopt.org/JuMP.jl/latest/style.html).

In [1]:
finite_strain_at(::Val{N}, v0::Float64, v::Float64) where {N} = -(3 * N + 2) / (3 * v) * finite_strain_at(Val(N - 1), v0, v)
finite_strain_at(::Val{1}, v0::Float64, v::Float64) = -1 / 3 / v * (v0 / v)^(2 / 3)
finite_strain_at(m::Int, v0::Float64, v::Float64) = finite_strain_at(Val(m), v0, v)

@time (finite_strain_at(100, 1.0, 9.0))
@time (finite_strain_at(100, 1.0, 9.0))
@time (finite_strain_at(101, 1.0, 9.0))
@time (finite_strain_at(101, 1.0, 9.0))

function finite_strain_at2(m::Int, reference_volume::Float64, v::Float64)
    m == 1 && return -(1 / 3 / v) * (reference_volume / v)
    -(3 * m + 2) / (3 * v) * finite_strain_at2(m - 1, reference_volume, v)
end

@time (finite_strain_at2(100, 1.0, 9.0))
@time (finite_strain_at2(100, 1.0, 9.0))
@time (finite_strain_at2(101, 1.0, 9.0))
@time (finite_strain_at2(101, 1.0, 9.0))

  0.321272 seconds (1.09 M allocations: 59.300 MiB, 5.30% gc time)
  0.000023 seconds (7 allocations: 208 bytes)
  0.005106 seconds (19.42 k allocations: 1.066 MiB)
  0.000063 seconds (7 allocations: 208 bytes)
  0.005170 seconds (19.45 k allocations: 1.079 MiB)
  0.000001 seconds (4 allocations: 160 bytes)
  0.000001 seconds (4 allocations: 160 bytes)
  0.000001 seconds (4 allocations: 160 bytes)


-2.11664381355344e63

### Writing type-stable code in julia

- If the return type can decided at compile time then it will be a *type stable*, i.e. if the return type is decided only by the types of the input[1]
- If the return type can’t decided until run time then it will be a *type unstable*, i.e. if the return type is decided by the values of the input, or by external factors[1]

Equivalently, a code is said to be *type-stable* if the type of every variable does not vary over time.

In [2]:
function sumofsins1(n::Int)  # type-unstable function
    r = 0  
    for i in 1:n
        r += sin(3.4)  
    end  
    return r  
end  

function sumofsins2(n::Int)  # type-stable function
    r = 0.0
    for i in 1:n
        r += sin(3.4)
    end  
    return r
end

sumofsins2 (generic function with 1 method)

In [3]:
sumofsins1(100000)
sumofsins2(100000)

@time [sumofsins1(100000) for i in 1:100];
@time [sumofsins2(100000) for i in 1:100];

  0.193035 seconds (302.06 k allocations: 15.074 MiB, 5.10% gc time)
  0.085354 seconds (122.84 k allocations: 6.347 MiB)


## OOP

In [2]:
struct Person{tag}
    name::String
    age::Int
    project::String
end

In [3]:
const Professor = Person{:professor}
const PhD = Person{:phd}
const Master = Person{:master};

In [4]:
prof = Professor("Somebody", 60, "many")
master = Master("Maria", 25, "great");

In [6]:
name(p::Person) = p.name
age(p::Person) = p.age
project(p::Person) = p.project;

In [7]:
project(master)

"great"

## References
1. [Julia Nomenclature](https://white.ucc.asn.au/2019/04/03/Julia-Nomenclature.html)